In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver

import getpass4


In [ ]:
load_dotenv(override=True)

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass4.getpass("Enter your Google AI API key: ")

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## Asynchronous LangGraph

### To run a tool:
- **Sync:** `tool.run(inputs)`
- **Async:** `await tool.arun(inputs)`

### To invoke the graph:
- **Sync:** `graph.invoke(state)`
- **Async:** `await graph.ainvoke(state)`


In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    
graph_builder = StateGraph(State)

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()

tool_search = Tool(
    name="search",
    description="Useful for when you need more information from an online search",
    func=serper.run,
)

In [ ]:
def calculate_math(expression: str) -> str:
    """Calculate mathematical expressions safely"""
    try:
        # Remove any potentially dangerous characters and evaluate safely
        allowed_chars: set[str] = set("0123456789+-*/(). ")
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"ผลลัพธ์: {expression} = {result}"
        else:
            return "ข้อผิดพลาด: นิพจน์ไม่ถูกต้อง ใช้ได้เฉพาะตัวเลขและเครื่องหมาย +, -, *, /, (, ) เท่านั้น"
    except Exception as e:
        return f"ข้อผิดพลาดในการคำนวณ: {str(e)}"


calculate_tool = Tool(
    name="calculator",
    func=calculate_math,
    description="Use this to calculate mathematical expressions. Input should be a valid math expression like '2 + 3 * 4' or '(10 + 5) / 3'",
)

calculate_tool.invoke("1+2")

## Next: Install Playwright

### On Windows and MacOS:
```bash
playwright install
```

### On Linux:
```bash
playwright install --with-deps chromium
```

Introducing nest_asyncio

Python async code only allows for one "event loop" processing asynchronous events.

The `nest_asyncio` library patches this, and is used for special situations, if you need to run a nested event loop.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

### The LangChain Community

One of the remarkable things about the rich LangChain community around it.

Check this out:

In [ ]:
import nest_asyncio; nest_asyncio.apply()

from playwright.async_api import async_playwright
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

async def setup(headless=False):
    p = await async_playwright().start()
    browser = await p.chromium.launch(headless=headless)
    # ใช้ from_browser โดยระบุ async_browser
    toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=browser)
    tools = toolkit.get_tools()
    return tools, browser, p

tools, browser, p = await setup(False)

# ... ใช้งาน tools ...

# ปิดงาน
# await browser.close()
# await p.stop()


In [ ]:
for tool in tools:
    print(f"{tool.name}={tool}")

In [ ]:
tools_dict = {tool.name: tool for tool in tools}

navigate_tool = tools_dict.get("navigate_browser")
extract_text_tool = tools_dict.get("extract_text")

await tools_dict["navigate_browser"].arun({"url": "https://cnn.com"})
text = await tools_dict["extract_text"].arun({})

# ctx = await browser.new_context(); ctx.set_default_navigation_timeout(60000)
# page = await ctx.new_page()
# await page.goto("https://www.cnn.com", wait_until="domcontentloaded")
# text = await tools_dict["extract_text"].arun({})

In [ ]:
import textwrap

print(textwrap.fill(text))

In [ ]:
all_tools = tools + [calculate_tool]

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

llm_with_tools = llm.bind_tools(all_tools)

def chatbot(state: State) -> State:
    """This function is called when the user sends a message to the chatbot."""

    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=all_tools))
graph_builder.add_conditional_edges(
    "chatbot", tools_condition, {"tools": "tools", "__end__": END}
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
config = {"configurable": {"thread_id": "222"}}

async def chat(user_input: str, history) -> str:
    """This function is called when the user sends a message to the chatbot."""
    result = await graph.ainvoke(
        {"messages": [{"role": "user", "content": user_input}]}, config=config
    )
    print(result)
    
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()